# Instructions

The places where you have enter code, to answer the questions, are marked with `# YOUR CODE HERE`. Once you have written your code you should remove any `raise NotImplementedError()` statements.


## Question 1 (4 points)

Suppose that we would like to minimize $f(x_1,x_2,x_3)=-5x_1-2x_1x_2 -3x_3$ which is subjected to the constraint $x_1+ x_2 + x_3 = 1$. 

Complete the function `constrained_bqm` so that it returns the corresponding BQM. Lagrange multiplier should be provided as a parameter.

Note: Name your variables in the format `x_i` and use BQM functionalities to incorporate the constraint.

The function `constrained_bqm` has

- Input:
    - `L`: a float representing the Lagrange multiplier
- Returns:
    - A BQM instance


In [45]:
%pip install dimod


Note: you may need to restart the kernel to use updated packages.


In [46]:
from dimod import BQM

def constrained_bqm(L):
    # YOUR CODE HERE

    # Define the objective function coefficients
    obj_coeffs = {('x1', 'x1'): -5, ('x1', 'x2'): -2, ('x3', 'x3'): -3}
    
    # Define the constraint coefficients
    constraint_coeffs = {('x1', 'x1'): L, ('x2', 'x2'): L, ('x3', 'x3'): L,
                         ('x1', 'x2'): L, ('x1', 'x3'): L, ('x2', 'x3'): L}
    
    # Create the BQM
    bqm = BQM(obj_coeffs, constraint_coeffs, 0, 'BINARY')
    
    # Define the linear biases for the constraint
    linear_constraints = {'x1': L, 'x2': L, 'x3': L}
    
    # Set the linear biases for the constraint
    for var, bias in linear_constraints.items():
        bqm.set_linear(var, bias)
    
    # raise NotImplementedError()
    # Do not modify anything below this line
    return bqm


In [47]:
# You can use this cell to call and check the output of the function

print(constrained_bqm(2))


BinaryQuadraticModel({'x1': 2.0, 'x2': 2.0, 'x3': 2.0, ('x1', 'x1'): -5.0, ('x1', 'x2'): -2.0, ('x3', 'x3'): -3.0}, {('x2', 'x1'): 2.0, ('x3', 'x1'): 2.0, ('x3', 'x2'): 2.0}, 0.0, 'BINARY')


In [48]:
# hidden tests will be used for grading.

In [49]:
# hidden tests will be used for grading.

## Question 2 (4 points)

Next questions are about the Knapsack Problem.

Given $n$ items, each with an associated weight $w_i$ and cost $c_i$, the goal of the Knapsack problem is to pack as many items to maximize the value of the packed knapsack, while not exceeding the knapsack capacity.

Let $x_i$ be a binary variable that is equal to 1 if $i$'th item is selected and 0 otherwise. 

In Knapsack problem, the objective is to maximize
$$
\sum c_ix_i
$$
subject to the constraint
$$
\sum w_ix_i \leq W. 
$$

However, note that when formulating a QUBO, we always consider the equivalent `minimization` problem. 

Complete the function `knapsack_objective` which takes as parameter a BQM instance and the list of costs and adds the objective function to be *minimized* to the input BQM.

Note: Name your variables in the format `x_i` and use BQM functionalities.

The function `knapsack_objective` has

- Inputs:
    - `bqm`: a BQM instance,
    - `costs`: a list of $n$ elements corresponding to costs of the items.
            
- Returns:
    - Modified `bqm`


In [50]:
from dimod import BQM

def knapsack_objective(bqm, costs):
    # YOUR CODE HERE

    # Get the number of items
    num_items = len(costs)
    
    # Add the objective function to the BQM
    for i in range(num_items):
        bqm.add_variable(f'x_{i}', -costs[i])

    # raise NotImplementedError()
    # Do not modify anything below this line
    return bqm


In [51]:
# You can use this cell to call and check the output of the function
from dimod import BQM

bqm = BQM("BINARY")
costs = [2, 7, 3, 4, 6]
bqm = knapsack_objective(bqm, costs)
print(bqm)


BinaryQuadraticModel({'x_0': -2.0, 'x_1': -7.0, 'x_2': -3.0, 'x_3': -4.0, 'x_4': -6.0}, {}, 0.0, 'BINARY')


In [52]:
# hidden tests will be used for grading.

In [53]:
# hidden tests will be used for grading.

## Question 3 (4 points)

Complete the function `knapsack_constraint` which takes as parameter a BQM instance and the list of costs, weights and capacity of the knapsack and adds the constraint to the input BQM.

Note: Name your variables in the format `x_i` and use BQM functionalities. Set Lagrange multiplier $L$ such that $L = \max\{c_i\} + 1$. Set `label="c"`.

The function `knapsack_constraint` has

- Inputs:
    - `bqm`: a BQM instance,
    - `weights`: a list of $n$ elements corresponding to the weights of the items,
    - `costs`-a list of $n$ elements corresponding to costs of the items,
    - `W`-capacity of Knapsack
            
- Returns:
    - Modified `bqm`


In [54]:
from dimod import BQM

def knapsack_constraint(bqm, weights, costs, W):
    # YOUR CODE HERE

    # Calculate Lagrange multiplier L
    L = max(costs) + 1
    
    # Add the constraint to the BQM
    for i in range(len(weights)):
        # Add the term for the weight constraint
        bqm.add_variable(f'x_{i}', -L * weights[i] * weights[i])
        
        # Add the terms for the Lagrange multiplier
        for j in range(i+1, len(weights)):
            bqm.add_interaction(f'x_{i}', f'x_{j}', 2 * L * weights[i] * weights[j])
    
    # Add the constant term
    bqm.offset += L * W * W

    # raise NotImplementedError()
    # Do not modify anything below this line
    return bqm


In [55]:
# You can use this cell to call and check the output of the function
from dimod import BQM

bqm = BQM("BINARY")
costs = [2, 7, 3, 4, 6]
W = 10
weights = [1, 4, 8, 2, 3]
bqm = knapsack_constraint(bqm, weights, costs, W)
print (bqm)


BinaryQuadraticModel({'x_0': -8.0, 'x_1': -128.0, 'x_2': -512.0, 'x_3': -32.0, 'x_4': -72.0}, {('x_1', 'x_0'): 64.0, ('x_2', 'x_0'): 128.0, ('x_2', 'x_1'): 512.0, ('x_3', 'x_0'): 32.0, ('x_3', 'x_1'): 128.0, ('x_3', 'x_2'): 256.0, ('x_4', 'x_0'): 48.0, ('x_4', 'x_1'): 192.0, ('x_4', 'x_2'): 384.0, ('x_4', 'x_3'): 96.0}, 800.0, 'BINARY')


In [56]:
# hidden tests will be used for grading.

In [57]:
# hidden tests will be used for grading.

## Question 4 (4 points)

Complete the function `knapsack_bqm` which takes as parameter the list of costs, weights and capacity of the knapsack and returns a BQM instance for the given knapsack problem.

Note: Name your variables in the format `x_i` and use the functions you have written previously.

The function `knapsack_bqm` has

- Inputs:
    - `weights`: a list of $n$ elements corresponding to the weights of the items,
    - `costs`-a list of $n$ elements corresponding to costs of the items,
    - `W`: capacity of Knapsack.
            
- Returns:
    - A BQM instance `bqm`.


In [58]:
from dimod import BQM

def knapsack_bqm(weights, costs, W):
    # YOUR CODE HERE

    # Create a BQM instance
    bqm = BQM('BINARY')
    
    # Add the objective function to the BQM
    bqm = knapsack_objective(bqm, costs)
    
    # Add the knapsack constraint to the BQM
    bqm = knapsack_constraint(bqm, weights, costs, W)

    # raise NotImplementedError()
    # Do not modify anything below this line
    return bqm


In [59]:
# You can use this cell to call and check the output of the function
from dimod import BQM
bqm = BQM("BINARY")
costs = [2, 7, 3, 4, 6]
W = 10
weights = [1, 4, 8, 2, 3]
bqm = knapsack_bqm(weights, costs, W)
print(bqm)


BinaryQuadraticModel({'x_0': -10.0, 'x_1': -135.0, 'x_2': -515.0, 'x_3': -36.0, 'x_4': -78.0}, {('x_1', 'x_0'): 64.0, ('x_2', 'x_0'): 128.0, ('x_2', 'x_1'): 512.0, ('x_3', 'x_0'): 32.0, ('x_3', 'x_1'): 128.0, ('x_3', 'x_2'): 256.0, ('x_4', 'x_0'): 48.0, ('x_4', 'x_1'): 192.0, ('x_4', 'x_2'): 384.0, ('x_4', 'x_3'): 96.0}, 800.0, 'BINARY')


In [60]:
# hidden tests will be used for grading.

In [61]:
# hidden tests will be used for grading.

## Question 5 (4 points)

Complete the function `anneal` which takes as parameter a BQM instance and number of reads and returns a sampleset object after running simulated annealing. Make the necessary imports.

The function `anneal` has

- Inputs:
    - `bqm`: a BQM instance,
    - `num_reads`: an integer coresponding to the number of reads for simulated annealing experiment.
            
- Returns:
    - A `SampleSet` instance 


In [62]:
from dimod import SimulatedAnnealingSampler

def anneal(bqm, num_reads):
    # YOUR CODE HERE

    # Initialize the simulated annealing sampler
    sampler = SimulatedAnnealingSampler()

    # Sample from the BQM using simulated annealing
    sampleset = sampler.sample(bqm, num_reads=num_reads)

    # raise NotImplementedError()
    # Do not modify anything below this line
    return sampleset


In [63]:
# You can use this cell to call and check the output of the function

linear = {'x1': 5, 'x2': -3}
quadratic = {('x1', 'x2'): 7}
offset = 2
vartype = 'BINARY'

bqm = BQM(linear, quadratic, offset, vartype)
sampleset = anneal(bqm, 1000)
print(sampleset)


    x1 x2 energy num_oc.
0    0  1   -1.0       1
1    0  1   -1.0       1
2    0  1   -1.0       1
3    0  1   -1.0       1
4    0  1   -1.0       1
5    0  1   -1.0       1
6    0  1   -1.0       1
7    0  1   -1.0       1
8    0  1   -1.0       1
9    0  1   -1.0       1
10   0  1   -1.0       1
11   0  1   -1.0       1
12   0  1   -1.0       1
13   0  1   -1.0       1
14   0  1   -1.0       1
15   0  1   -1.0       1
16   0  1   -1.0       1
17   0  1   -1.0       1
18   0  1   -1.0       1
19   0  1   -1.0       1
20   0  1   -1.0       1
21   0  1   -1.0       1
22   0  1   -1.0       1
23   0  1   -1.0       1
24   0  1   -1.0       1
25   0  1   -1.0       1
26   0  1   -1.0       1
27   0  1   -1.0       1
28   0  1   -1.0       1
29   0  1   -1.0       1
30   0  1   -1.0       1
31   0  1   -1.0       1
32   0  1   -1.0       1
33   0  1   -1.0       1
34   0  1   -1.0       1
35   0  1   -1.0       1
36   0  1   -1.0       1
37   0  1   -1.0       1
38   0  1   -1.0       1


In [64]:
# hidden tests will be used for grading.

In [65]:
# hidden tests will be used for grading.

## Question 6 (4 points)

Complete the function `knapsack_solution` which takes as parameter number of reads, the list of costs, weights and weight of the knapsack and returns the sample with the lowest energy for the given Knapsack instance.

Note: You should use the functions `anneal` and `knapsack_bqm` you have written previously.

The function `knapsack_solution` has

- Inputs:
    - `num_reads`: number of reads,
    - `weights`: a list of $n$ elements corresponding to the weights of the items,
    - `costs`-a list of $n$ elements corresponding to costs of the items,
    - `W`-capacity of Knapsack

- Returns:
    - `solution`: a tuple consisting of solution vector and energy


In [66]:
from dimod import BQM
from dimod.reference.samplers import SimulatedAnnealingSampler


def knapsack_solution(num_reads,weights, costs, W):
    # YOUR CODE HERE

    # Create the BQM instance for the Knapsack problem
    bqm = knapsack_bqm(weights, costs, W)
    
    # Anneal the BQM and get the lowest energy sample
    response = anneal(bqm, num_reads=num_reads)

    # Get the sample with the lowest energy
    sample = response.first.sample
    energy = response.first.energy

    solution = energy

    # raise NotImplementedError()
    # Do not modify anything below this line
    return solution


In [67]:
# You can use this cell to call and check the output of the function

costs = [2, 7, 3, 4, 6]
W = 10
weights = [1, 4, 8, 2, 3]
num_reads = 100
solution = knapsack_solution(num_reads,weights, costs, W)
print(solution)


285.0


In [68]:
# hidden tests will be used for grading.